In [ ]:
import csv
import time
import board
import busio
import adafruit_ads1x15.ads1015 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import adafruit_bme280
import RPi.GPIO as GPIO
import adafruit_ltr390

# Setup GPIO for the strain gauge
GPIO.setmode(GPIO.BOARD)
GPIO.setup(11, GPIO.IN)

# Initialize I2C bus
i2c = busio.I2C(board.SCL, board.SDA)

# Initialize BME280 sensor
bme280 = adafruit_bme280.Adafruit_BME280_I2C(i2c)

# Initialize ADS1015 ADC
ads = ADS.ADS1015(i2c)
chan1 = AnalogIn(ads, ADS.P0) # connect to fuel flow sensor
chan2 = AnalogIn(ads, ADS.P1) # connect to oil pressure sensor

# Initialize LTR-390 optical sensor
ltr390 = adafruit_ltr390.LTR390(i2c)

# Initialize variables
n_samples = 1000  # number of data samples to read
sample_rate = 1  # sample rate in seconds

# Initialize data list
data = []

# Read and log data
for i in range(n_samples):
    # Read data from sensors
    time_stamp = time.time()
    strain_gauge = GPIO.input(11)
    pressure = bme280.pressure
    temperature = bme280.temperature
    oil_pressure = chan2.voltage
    fuel_flow = chan1.voltage
    light_level = ltr390.light
    
    # Categorize performance based on conditions
    if oil_pressure > 4.5:
        performance = 'Excellent'
    elif oil_pressure > 4.0:
        performance = 'Good'
    elif oil_pressure > 3.5:
        performance = 'Fair'
    else:
        performance = 'Poor'
    
    # Add data to list
    data.append([time_stamp, strain_gauge, pressure, temperature, oil_pressure,fuel_flow, light_level, performance])
    
    # Wait for next sample
    time.sleep(sample_rate)

# Open CSV file
with open('sensor_data.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Time', 'Strain Gauge', 'Pressure Sensor', 'Temperature Sensor', 'Oil Pressure Sensor','Fuel Flow Sensor', 'Light Sensor', 'Performance']
    writer = csv.writer(csv_file)
    writer.writerow(fieldnames)

    # Write data to CSV file
    for row in data:
        writer.writerow(row)




Part 1 explanation:
    This code snippet reads data from various sensors (strain gauge, BME280, ADS1015, and LTR-390) and categorizes the performance based on the oil pressure readings. The data is logged in a list and then written to a CSV file. The code also sets the sample rate and number of data samples to be collected

In [ ]:
import pandas as pd
import xgboost as xgb

# Load CSV file into a pandas DataFrame
df = pd.read_csv('sensor_data.csv')

# Split data into features (X) and target (y)
X = df.iloc[:, 1:-1] # ignore the first column (time) and last column (performance)
y = df.iloc[:, -1] # target is the last column (performance)

# Train XGBoost model
model = xgb.XGBClassifier()
model.fit(X, y)

# Predict performance using XGBoost model
y_pred = model.predict(X)

# Add predicted performance to DataFrame
df['predicted_performance'] = y_pred

# Save DataFrame to new CSV file
df.to_csv('sensor_data_with_predictions.csv', index=False)


Part 2 Explanation:
    This code loads a CSV file containing sensor data into a pandas DataFrame, splits the data into features (X) and target (y), and trains an XGBoost classification model on the data. The model is then used to predict the performance of the sensor based on the features, and the predicted performance is added to the DataFrame. Finally, the updated DataFrame is saved to a new CSV file.

In [ ]:
import pandas as pd
import lightgbm as lgb

# Load CSV file with predicted performance into a pandas DataFrame
df = pd.read_csv('sensor_data_with_predictions.csv')

# Split data into features (X) and target (y)
X = df.iloc[:, 1:-2] # ignore the first column (time), last two columns (performance and predicted_performance)
y = df.iloc[:, -2] # target is the second-to-last column (predicted_performance)

# Train LightGBM model
model = lgb.LGBMClassifier()
model.fit(X, y)

# Classify if damaged or low-performing part needs replacement using LightGBM model
y_pred = model.predict(X)

# Add replacement needs to DataFrame
df['replacement_needed'] = y_pred

# Save DataFrame to new CSV file
df.to_csv('sensor_data_with_predictions_and_replacement.csv', index=False)


Part 3 Explanation:
    This code loads a CSV file containing sensor data and a column with predicted performance using an XGBoost model. It then trains a LightGBM model using the same dataset, predicts whether a damaged or low-performing part needs replacement, adds this information to the DataFrame, and saves the updated DataFrame to a new CSV file.



